## Trabajo fin de máster: Santander Bikes (análisis y predicción)

Este notebook contiene todo el código realizado, del cual se obtienen los gráficos y las métricas incluidas en el informe en PDF. Dado que no se ha seleccionado una semilla, los resultados pueden variar sensiblemente, pero nunca de manera que alteren las conclusiones, pues el trabajo se ha ejecutado varias veces y las salidas apenas han diferido por unas pocas cifras decimales. El código está comentado y se explica sucintamente la utilidad de la mayoría de las celdas. Sin embargo, la explicación detallada de las salidas y la interpretación se encuentran únicamente en el informe. Sirva esto, pues, como anexo.

In [ ]:
from datetime import datetime
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
import datetime
import os
from pandas_profiling import ProfileReport
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import RandomizedSearchCV
import collections

In [ ]:
# os.chdir("E:/pythonProject") he seleccionado este directorio de trabajo. Comentar celda si no es necesario usarla

Cargamos el archivo y renombramos las columnas para mayor claridad en la interpretación:

In [ ]:
raw = pd.read_csv("../input/london-bike-sharing-dataset/london_merged.csv")
raw = raw.rename(columns = {'cnt': 'Count', 't1': 'Temperature', 't2': 'Feels Like', 'hum': 'Humidity', 'wind_speed': 'Wind Speed',
                           'weather_code':'Weather Code', 'is_holiday':'Holiday', 'is_weekend':'Weekend', 'season':'Season'})
raw = raw.reset_index()

In [ ]:
raw

Cambiamos los valores de las variables que son categóricas:

In [ ]:
# raw['Holiday'] = raw['Holiday'].replace(0, False)
# raw['Holiday'] = raw['Holiday'].replace(1, True)
# raw['Holiday'] = raw['Holiday'].astype('bool')

# raw['Weekend'] = raw['Weekend'].replace(0, False)
# raw['Weekend'] = raw['Weekend'].replace(1, True)
# raw['Weekend'] = raw['Weekend'].astype('bool')

raw['Season'] = raw['Season'].replace({0: 'Spring', 1: 'Summer', 2: 'Autumn', 3: 'Winter'})
raw['Weather Code'] = raw['Weather Code'].replace({1: 'Clear', 2: 'Few Clouds', 3: 'Broken Clouds', 4: 'Cloudy',
                                                  7: 'Light Rain', 10: 'Thunderstorm', 26: 'Snowfall', 94: 'Freezing Fog'})

raw['timestamp'] = pd.to_datetime(raw['timestamp'])
raw.dtypes

Aquí extraigo nuevas variables a partir de "timestamp": año, mes, día, tramo horario (mañana, tarde, noche)

In [ ]:
raw['hour'] = raw['timestamp'].apply(lambda time: time.hour) 
raw['month'] = raw['timestamp'].apply(lambda time: time.month)
raw['day_of_week'] = raw['timestamp'].apply(lambda time: time.dayofweek)

# Renombramos los días de la semana
date_names = {0: 'Mon', 1: 'Tue', 2: 'Wed', 3: 'Thu', 4: 'Fri', 5: 'Sat', 6: 'Sun'} 
raw['day_of_week'] = raw['day_of_week'].map(date_names)

raw.drop('timestamp', axis = 1, inplace = True)

In [ ]:
raw

Hacemos un análisis inicial de los datos usando ProfileReport:

In [ ]:
profile = ProfileReport(raw)

In [ ]:
profile

In [ ]:
# Creación del dataset bueno

london = raw.join(pd.get_dummies(raw['Weather Code']), on = raw['df_index']).drop(columns = ['Weather Code'])\
.join(pd.get_dummies(raw['Season']), on = raw['df_index']).drop(columns = ['Season'])\
.join(pd.get_dummies(raw['day_of_week']), on = raw['df_index']).drop(columns = ['day_of_week'])\
.drop(columns = ['df_index'])

In [ ]:
results = london['Count'] #variable objetivo
features = london.drop(columns = ['Count']) #variables independientes

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, results, test_size = 0.20, shuffle = False)

#### REGRESIÓN LINEAL

In [ ]:
lr = LinearRegression()
lr.fit(x_train, y_train)

In [ ]:
lr.score(x_test, y_test)

In [ ]:
print("regresión lineal: ", mean_absolute_error(lr.predict(x_test), y_test),
      "\nPrediciendo la media: ", mean_absolute_error(np.ones([y_test.shape[0]]) * np.mean(y_test), y_test))

In [ ]:
error_lr = y_test - lr.predict(x_test)

In [ ]:
plt.hist(error_lr, bins = np.arange(-2000, 2000, 50)) #error sin normalizar

plt.title('Error regresión lineal (en núm. bicis)')
plt.xlabel('Error')
plt.ylabel('Frecuencia')

Nuestra regresión lineal parece ligeramente mejor que predecir únicamente la media para todas las observaciones test, pero no es un gran avance. Probemos con otros modelos.

#### SVR

In [ ]:
stdscl = StandardScaler()
std_x_train = stdscl.fit_transform(x_train.values)
std_x_test = stdscl.fit_transform(x_test.values)

In [ ]:
svr = SVR()
svr.fit(std_x_train, y_train)

In [ ]:
print("SVR: ", mean_absolute_error(svr.predict(std_x_test), y_test),
      "\nPrediciendo la media: ", mean_absolute_error(np.ones([y_test.shape[0]]) * np.mean(y_test), y_test))

In [ ]:
error_svr = y_test - svr.predict(std_x_test)

In [ ]:
plt.hist(error_svr, bins = np.arange(-1000, 4600, 50)) #error sin normalizar

plt.title('Error SVR (en núm. bicis)')
plt.xlabel('Error')
plt.ylabel('Frecuencia')

Prácticamente el mismo resultado.

#### NB

In [ ]:
brr = BayesianRidge()
brr.fit(x_train, y_train)

In [ ]:
print("Naive Bayes: ", mean_absolute_error(brr.predict(x_test), y_test),
      "\nPrediciendo la media: ", mean_absolute_error(np.ones([y_test.shape[0]]) * np.mean(y_test), y_test))

In [ ]:
error_brr = y_test - brr.predict(x_test)

In [ ]:
plt.hist(error_brr, bins = np.arange(-2000, 4600, 50)) #error sin normalizar

plt.title('Error NB (en núm. bicis)')
plt.xlabel('Error')
plt.ylabel('Frecuencia')

Seguimos sin hacer progresos.

#### RF

In [ ]:
rf = RandomForestRegressor()
rf.fit(x_train, y_train)

In [ ]:
rf.score(x_test, y_test) #En realidad, esta métrica no es muy importante, aunque una puntuación alta es alentadora.

In [ ]:
print("Random Forest: ", mean_absolute_error(rf.predict(x_test), y_test),
      "\nPrediciendo la media: ", mean_absolute_error(np.ones([y_test.shape[0]]) * np.mean(y_test), y_test))

In [ ]:
london['Count'].max()

In [ ]:
plt.hist(london['Count'], bins = np.arange(0, 6500, 100))

plt.title('Distribución de la variable objetivo')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')

In [ ]:
rf.predict(x_test)

In [ ]:
error = y_test - rf.predict(x_test)

In [ ]:
plt.hist(error, bins = np.arange(-1500, 1500, 50)) #error sin normalizar

plt.title('Distribución del error de la predicción (RF)')
plt.xlabel('Error')
plt.ylabel('Frecuencia')

In [ ]:
error.mean(), error.std() #no está mal pero tenemos mucha desviación estándar

In [ ]:
error_norm = error/y_test
error_norm

In [ ]:
plt.hist(error_norm, bins = np.arange(-3, error_norm.max(), 0.1))

plt.title('Distribución del error normalizado de la predicción (RF)')
plt.xlabel('Error')
plt.ylabel('Frecuencia')

In [ ]:
error_norm.mean(), error_norm.std()

In [ ]:
plt.scatter(y_test, error_norm) #el modelo se equivoca algo más cuanto más pequeño es el número de bicis que se cogen

plt.title('Error normalizado vs Número de bicis')
plt.xlabel('Número de bicis')
plt.ylabel('Error')

In [ ]:
plt.scatter(rf.predict(x_test), y_test, alpha = 0.5)
plt.plot(y_test, y_test, color = 'magenta')

Parece un modelo bastante bueno cuando hay muchas bicis pero "no tan bueno" cuando hay pocas. Centrémonos ahora en las observaciones con pocas bicis:

In [ ]:
y_test[y_test < 1000]

In [ ]:
rf.predict(x_test)[y_test < 1000]

In [ ]:
error_pequeño = y_test[y_test < 1000] - rf.predict(x_test)[y_test < 1000]

In [ ]:
error_pequeño.mean()

In [ ]:
(y_test[y_test < 1000] - y_test[y_test < 1000].mean()).mean() #en recuentos pequeños parece "mucho" mejor predecir la media

In [ ]:
plt.scatter(y_test[y_test < 1000], rf.predict(x_test)[y_test < 1000])

plt.xlabel('Número de bicis real')
plt.ylabel('Predicción')
plt.title('Predicción Random Forest vs Valores reales')

In [ ]:
plt.scatter(y_test[y_test < 1000], error_pequeño)

plt.xlabel('Número de bicis')
plt.ylabel('Error')
plt.title('Error Random Forest a lo largo de las observaciones')

Estas predicciones han sido hechas a partir de un Random Forest sin ningún tuneo. Descubramos sus parámetros para ver si es posible tunear alguno de ellos y mejorar la precisión del modelo.

In [ ]:
rf.get_params() #base para ver qué puedo tunear

In [ ]:
rf.feature_importances_

In [ ]:
importances = rf.feature_importances_
indices = np.argsort(importances)
feature_names = features.keys()

plt.title('Importancia de variables')
plt.barh(range(len(indices)), importances[indices], color = 'magenta', align = 'center')
plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
plt.xlabel('Importancia relativa')
plt.show()

Parece que la hora es la variable más importante con mucha diferencia. ¿Es eso verdad? Puede que nuestro modelo no esté ponderando correctamente. Para ver si el modelo funciona bien, vamos a añadir una variable aleatoria que llamaremos "random" y que, como cabe esperar, no aportará absolutamente nada al modelo. Si el modelo está bien construido, esta variable tendrá una importancia nula o casi nula.

In [ ]:
# añado una columna random

london['random'] = np.random.random(london['Clear'].size) #que tenga la misma longitud que las otras columnas!

In [ ]:
results_2 = london['Count'] #variable objetivo
features_2 = london.drop(columns = ['Count']) #variables independientes

In [ ]:
x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(features_2, results_2, test_size = 0.20, shuffle = False)

In [ ]:
rf_2 = RandomForestRegressor()
rf_2.fit(x_train_2, y_train_2)

In [ ]:
rf_2.score(x_test_2, y_test_2)

In [ ]:
print("Random Forest: ", mean_absolute_error(rf_2.predict(x_test_2), y_test_2),
      "\nPrediciendo la media: ", mean_absolute_error(np.ones([y_test.shape[0]]) * np.mean(y_test_2), y_test_2))

In [ ]:
importances_2 = rf_2.feature_importances_
indices_2 = np.argsort(importances_2)
feature_names_2 = features_2.keys()

plt.title('Importancia de variables')
plt.barh(range(len(indices_2)), importances_2[indices_2], color = 'magenta', align = 'center')
plt.yticks(range(len(indices_2)), [feature_names_2[i] for i in indices_2])
plt.xlabel('Importancia relativa')
plt.show()

Hemos visto que la hora parece muy importante, y que muchas variables no aportan ninguna información. ¿Podríamos construir un modelo más simple sin perder información? Veamos si haciendo una selección de variables podemos conseguirlo.

In [ ]:
dict_no_ordenado = dict(zip(rf.feature_importances_, x_train.columns))

In [ ]:
ordered = collections.OrderedDict(sorted(dict_no_ordenado.items()))

In [ ]:
ordered

In [ ]:
selected_features = np.array(list(ordered.values()))[-1:-15:-1]

In [ ]:
rf.fit(x_train.loc[:,selected_features], y_train)

In [ ]:
rf.score(x_test.loc[:, selected_features], y_test)

In [ ]:
print("Random Forest: ", mean_absolute_error(rf.predict(x_test.loc[:,selected_features]), y_test),
      "\nPrediciendo la media: ", mean_absolute_error(np.ones([y_test.shape[0]]) * np.mean(y_test), y_test))

Parece que eliminar todas esas variables no afecta de manera significativa al modelo. Veamos a continuación una distribución de nuestra variable objetivo en función de la hora:

In [ ]:
plt.scatter(london['hour'], london['Count'])

plt.xlabel('Hora')
plt.ylabel('Número de bicis')
plt.title('Número de bicis vs Hora del día')

No parece haber una relación muy lineal entre la hora y el número de bicis (casi todo al ir a trabajar o al salir del trabajo), pero se observa claramente que existe una razón para que esta variable sea tan importante.

In [ ]:
# Random Forest con tuneo de parámetros
# Número de árboles
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 1000, num = 10)]
# Tuneamos el número de variables a usar en cada caso
max_features = ['auto', 'sqrt']
# Máximo número de niveles en cada árbol
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Sampsize mínimo para cada nodo
min_samples_split = [2, 5, 10]
# Sampsize mínimo para cada hoja del árbol
min_samples_leaf = [1, 2, 4]
# Método de selección
bootstrap = [True, False]
# Creamos la rejilla con los valores de arriba
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
rf_3 = RandomForestRegressor()
# Búsqueda aleatoria de parámetros con validación cruzada, 
# buscamos entre muchas combinaciones distintas.
rf_random = RandomizedSearchCV(estimator = rf_3, param_distributions = random_grid,
                               n_iter = 100, cv = 3, verbose = 3,
                               n_jobs = 5) #n_jobs = número de núcleos del procesador (cambiar en función del ordenador)
rf_random.fit(x_train, y_train)

In [ ]:
print("Random Forest tuneado: ", mean_absolute_error(rf_random.predict(x_test), y_test),
      "\nPrediciendo la media: ", mean_absolute_error(np.ones([y_test.shape[0]]) * np.mean(y_test), y_test))

In [ ]:
rf_random.best_params_

In [ ]:
rf_random.predict(x_test)

In [ ]:
error_random = y_test - rf_random.predict(x_test)

In [ ]:
plt.hist(error_random, bins = np.arange(-1500, 1500, 50)) #error sin normalizar

plt.title('Distribución del error de la predicción (RF)')
plt.xlabel('Error')
plt.ylabel('Frecuencia')

In [ ]:
error_random.mean(), error_random.std() #seguimos con mucha desviación estándar

In [ ]:
error_random_norm = error_random/y_test
error_random_norm

In [ ]:
plt.hist(error_random_norm, bins = np.arange(-3, error_random_norm.max(), 0.1))

plt.title('Distribución del error normalizado de la predicción (RF)')
plt.xlabel('Error')
plt.ylabel('Frecuencia')

In [ ]:
error_random_norm.mean(), error_random_norm.std()

In [ ]:
plt.scatter(y_test, error_random_norm, alpha = 0.3)

plt.title('Error normalizado vs Número de bicis')
plt.xlabel('Número de bicis')
plt.ylabel('Error')

In [ ]:
plt.scatter(rf_random.predict(x_test), y_test, alpha = 0.5)
plt.plot(y_test, y_test, color = 'magenta') #de momento parece que el tuneo no está siendo exitoso

Parece un modelo bastante bueno cuando hay muchas bicis pero "no tan bueno" cuando hay pocas. Centrémonos ahora en las observaciones con pocas bicis:

In [ ]:
y_test[y_test < 1000]

In [ ]:
rf_random.predict(x_test)[y_test < 1000]

In [ ]:
error_pequeño_random = y_test[y_test < 1000] - rf_random.predict(x_test)[y_test < 1000]

In [ ]:
error_pequeño_random.mean()

In [ ]:
(y_test[y_test < 1000] - y_test[y_test < 1000].mean()).mean() #Sigue siendo mejor predecir la media en recuentos pequeños

In [ ]:
plt.scatter(y_test[y_test < 1000], rf_random.predict(x_test)[y_test < 1000])
plt.plot(y_test[y_test < 1000], y_test[y_test < 1000], color = "magenta")

plt.xlabel('Número de bicis real')
plt.ylabel('Predicción')
plt.title('Predicción Random Forest vs Valores reales')

In [ ]:
plt.scatter(y_test[y_test < 1000], error_pequeño_random)
plt.axhline(y=0, color='m', linestyle='-')

plt.xlabel('Número de bicis')
plt.ylabel('Error')
plt.title('Error Random Forest a lo largo de las observaciones')

### Ideas de mejora para el futuro:

#### - mejorar los datos (hacer más features que estén mejor) O mejorar los modelos (tunear parámetros del RF)

#### - probar a entrenar el modelo con pocos datos e ir subiendo: ¿a partir de qué punto el aprendizaje del modelo "se estanca"?

#### - buscar más"métricas de calidad" para ver si mi modelo es bueno en las cosas que importan

#### - probar una red neuronal y entrenarla con datos en vivo